In [125]:
Assignment:4
1. you have to create one supervisor node.
2. create one router function
3. create three more node
3.1 llm call (llm node)
3.2 RAG (rag node)
3.3 web crawler(fetch the info in realtime from internet)
4. created one more node after this for validation for generated output --> explore the validation part how to do that
5. if validation going to be failed in that case again go to supervioser node and then supervisor node will again decide what needs to be call next
6. once the validation will pass then only generate the final output

Supervisor ➝ Router ➝ (LLM / RAG / Web) ➝ ✅ Pydantic Validation ➝ Post-Validation Router ➝ (Final Output / Supervisor Retry)


SyntaxError: invalid character '➝' (U+279D) (1599413519.py, line 12)

#### Step 1. Import Libraries

In [1]:
import operator
from typing import List
from pydantic import BaseModel , Field
from langchain.prompts import PromptTemplate
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph,END

#### Step 2. Load env

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

#### Step 4. Init o1-mini Model

In [4]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="o1-mini")
result = model.invoke("Hi")
print(result)

content='Hello! How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 340, 'prompt_tokens': 9, 'total_tokens': 349, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'o1-mini-2024-09-12', 'system_fingerprint': 'fp_3da8b0b088', 'id': 'chatcmpl-BhIzeZMtVEntidgA2bR7fpCzxhjah', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--a9297e3c-5c47-46b9-9467-53603a9b7a6d-0' usage_metadata={'input_tokens': 9, 'output_tokens': 340, 'total_tokens': 349, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 320}}


#### Step 5. Embeddings

In [9]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en")
len(embeddings.embed_query("Elo, mate"))

384

#### Step 6: Create Vector Database

In [10]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_community.vectorstores import Chroma 
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [11]:
loader = TextLoader(r"C:\Users\aditi_zg48xl\Downloads\Agentic AI\Assignment_4\Data\genetic_diseases.txt")
docs = loader.load()

In [12]:
docs 

[Document(metadata={'source': 'C:\\Users\\aditi_zg48xl\\Downloads\\Agentic AI\\Assignment_4\\Data\\genetic_diseases.txt'}, page_content="Genetic diseases are medical conditions caused by anomalies in the genome, either inherited from one's parents or acquired through spontaneous mutations. These diseases can be categorized based on the type and location of the genetic defect: single-gene disorders, multifactorial disorders, chromosomal abnormalities, and mitochondrial genetic diseases.\n\nSingle-gene disorders, or monogenic diseases, result from mutations in a single gene. These include well-known conditions such as cystic fibrosis (CF), sickle cell anemia, Tay-Sachs disease, and Huntingtonâ€™s disease. In CF, mutations in the CFTR gene cause defective chloride ion transport, leading to thickened mucus in the lungs and digestive system. Sickle cell anemia, on the other hand, arises from a point mutation in the HBB gene, causing abnormal hemoglobin structure and leading to hemolytic ane

#### Step 7.1 Using recursive text splitter divide into chunks

In [13]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 50
)

In [14]:
new_docs = text_splitter.split_documents(documents = docs)

In [15]:
new_docs

[Document(metadata={'source': 'C:\\Users\\aditi_zg48xl\\Downloads\\Agentic AI\\Assignment_4\\Data\\genetic_diseases.txt'}, page_content="Genetic diseases are medical conditions caused by anomalies in the genome, either inherited from one's parents or acquired through spontaneous mutations. These diseases can be categorized based on the"),
 Document(metadata={'source': 'C:\\Users\\aditi_zg48xl\\Downloads\\Agentic AI\\Assignment_4\\Data\\genetic_diseases.txt'}, page_content='These diseases can be categorized based on the type and location of the genetic defect: single-gene disorders, multifactorial disorders, chromosomal abnormalities, and mitochondrial genetic diseases.'),
 Document(metadata={'source': 'C:\\Users\\aditi_zg48xl\\Downloads\\Agentic AI\\Assignment_4\\Data\\genetic_diseases.txt'}, page_content='Single-gene disorders, or monogenic diseases, result from mutations in a single gene. These include well-known conditions such as cystic fibrosis (CF), sickle cell anemia, Tay-Sachs 

#### Step 7.2 Using FAISS VectorDB

In [16]:
from langchain_community.vectorstores import FAISS

vector_db = FAISS.from_documents(new_docs, embeddings)


In [17]:
retriever = vector_db.as_retriever(search_kwargs={"k" : 5})

In [18]:
retriever.invoke("What are simple-gene disorders")

[Document(id='dfd2db0b-f852-4b9e-a59d-2c5a532aa2d9', metadata={'source': 'C:\\Users\\aditi_zg48xl\\Downloads\\Agentic AI\\Assignment_4\\Data\\genetic_diseases.txt'}, page_content='Single-gene disorders, or monogenic diseases, result from mutations in a single gene. These include well-known conditions such as cystic fibrosis (CF), sickle cell anemia, Tay-Sachs disease, and'),
 Document(id='6046c8d8-b10c-43b4-8148-75296300e58b', metadata={'source': 'C:\\Users\\aditi_zg48xl\\Downloads\\Agentic AI\\Assignment_4\\Data\\genetic_diseases.txt'}, page_content='challenge in predicting and managing these disorders.'),
 Document(id='a01afaca-f679-432e-91be-87a847d728aa', metadata={'source': 'C:\\Users\\aditi_zg48xl\\Downloads\\Agentic AI\\Assignment_4\\Data\\genetic_diseases.txt'}, page_content="Genetic diseases are medical conditions caused by anomalies in the genome, either inherited from one's parents or acquired through spontaneous mutations. These diseases can be categorized based on the"),
 

In [19]:
from pydantic import BaseModel, Field

In [20]:
from pydantic import BaseModel, Field

class RoutingOutput(BaseModel):
    Topic: str = Field(description="Must be one of 'VectorDB', 'LLM', or 'WebSearch'")

In [21]:
from langchain.output_parsers import PydanticOutputParser

In [22]:
parser = PydanticOutputParser(pydantic_object=RoutingOutput)

In [23]:
parser.get_format_instructions

<bound method PydanticOutputParser.get_format_instructions of PydanticOutputParser(pydantic_object=<class '__main__.RoutingOutput'>)>

In [24]:
class AgentState(TypedDict):
    message: Annotated[Sequence[BaseModel],operator.add]

In [25]:
AgentState = {}

In [26]:
AgentState["messages"]=[]

In [27]:
AgentState

{'messages': []}

In [28]:
AgentState["messages"].append("Hi, how are you?")

In [29]:
AgentState["messages"].append("Supp")

In [30]:
AgentState["messages"].append("Reply to me")

In [31]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [32]:
def function_1(state: AgentState):
    question = state["messages"][-1]
    print("Question:", question)

    template = """
    Your task is to classify the user query into one of the following categories:
    [RAG, LLM, WebSearch].

    Only respond with the category name and nothing else.

    User query: {question}
    {format_instructions}
    """

    prompt = PromptTemplate(
        template=template,
        input_variables=["question"],
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )

    chain = prompt | model | parser

    response = chain.invoke({"question": question})
    print("Parsed response:", response)

    return {"messages": [response.Topic]}


In [33]:
state={"messages":["What are simple-gene disorders"]}

In [34]:
function_1(state)

Question: What are simple-gene disorders
Parsed response: Topic='WebSearch'


{'messages': ['WebSearch']}

In [35]:
# def router(state: AgentState):
#     print("->-> Router ->->")
    
#     topic = state["messages"][-1].strip().lower()
#     print("Routing topic:", topic)

#     if topic == "vectordb":
#         return "RAG"
#     elif topic == "llm":
#         return "LLM Call"
#     elif topic == "websearch":
#         return "Web Crawler"
#     else:
#         print("Unknown topic, defaulting to LLM Call")
#         return "LLM Call"

def router(state: AgentState):
    print("->-> Router ->->")
    
    topic = state["messages"][-1].strip().lower()
    print("Routing topic:", topic)

    if topic == "vectordb":
        return {"next": "RAG"}
    elif topic == "llm":
        return {"next": "LLM Call"}
    elif topic == "websearch":
        return {"next": "Web Crawler"}
    else:
        print("Unknown topic, defaulting to LLM Call")
        return {"next": "LLM Call"}



In [36]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [37]:
# # RAG Function
# def function_2(state:AgentState):
#     print("-> RAG Call ->")
    
#     question = state["messages"][0]
    
#     prompt=PromptTemplate(
#         template="""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:""",
        
#         input_variables=['context', 'question']
#     )
    
#     rag_chain = (
#         {"context": retriever | format_docs, "question": RunnablePassthrough()}
#         | prompt
#         | model
#         | StrOutputParser()
#     )
#     result = rag_chain.invoke({"question": {"question": question}})
#     return  {"messages": [result]}

def function_2(state: AgentState):
    print("-> RAG Call ->")

    question = state["messages"][0]

    prompt = PromptTemplate(
        template=(
            "You are an assistant for question-answering tasks. Use the following pieces of retrieved context "
            "to answer the question. If you don't know the answer, just say that you don't know. "
            "Use three sentences maximum and keep the answer concise.\n"
            "Question: {question}\nContext: {context}\nAnswer:"
        ),
        input_variables=["context", "question"]
    )

    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
    )

    # Only pass the question as a string, not as {"question": question}
    result = rag_chain.invoke(question)

    return {"messages": [result]}


In [38]:
def function_3(state: AgentState):
    print("->-> LLM Call ->->")

    question = state["messages"][0]

    complete_query = (
        "Answer the following question with your real-world knowledge. "
        f"User question: {question}"
    )

    response = model.invoke(complete_query)
    print("LLM Raw Response:", response)

    # Extract answer content safely
    if hasattr(response, 'content') and response.content is not None:
        answer = response.content
    elif hasattr(response, 'text'):
        answer = response.text
    else:
        answer = str(response)

    return {"messages": [answer]}


In [39]:
from langchain_community.tools import DuckDuckGoSearchRun

In [40]:
search = DuckDuckGoSearchRun()

In [41]:
def function_4(state: AgentState):
    print("->-> Web Search Call ->->")

    question = state["messages"][0]

    # Run the web search
    results = search.run(question)
    
    print("Search Result:", results)
    
    return {"messages": [results]}

In [42]:
from langgraph.graph import StateGraph, END

In [43]:
from pydantic import BaseModel, ValidationError, validator

# Define a model for validating the response
class OutputModel(BaseModel):
    response: str

    @validator("response")
    def check_min_length_and_topic(cls, value):
        if not value or len(value.strip()) < 1:
            raise ValueError("Output too short.")
        if "genetic" not in value.lower() and "disease" not in value.lower():
            raise ValueError("Output does not relate to genetic diseases.")
        return value

# Agentic validation function
def function_5(state):
    topic = state.get("topic", "").lower()
    response = state.get("response", "")

    # If topic is "llm", bypass strict validation
    if topic == "llm":
        if response.strip():
            state["validated"] = True
        else:
            state["validated"] = False
            state["validation_error"] = "Empty response for LLM"
        return state

    # For other topics, do stricter checks
    try:
        if not response.strip():
            raise ValueError("Missing or empty response.")
        if topic == "genetic diseases" and "genetic" not in response.lower() and "disease" not in response.lower():
            raise ValueError("Output does not relate to genetic diseases.")
        state["validated"] = True
    except Exception as e:
        state["validated"] = False
        state["validation_error"] = str(e)

    return state



C:\Users\aditi_zg48xl\AppData\Local\Temp\ipykernel_11636\895920501.py:7: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator("response")


In [44]:
def post_validation_router(state):
    if state.get("validated"):
        return "Final Output"
    else:
        return "Supervisor"



In [45]:
def final_output(state: AgentState):
    print("✅ Final Output Node")
    
    # The actual answer is second last because the last message is "VALIDATED"
    answer = state["messages"][-2] if state["messages"][-1] == "VALIDATED" else state["messages"][-1]
    
    return {"messages": [f"Final Answer: {answer}"]}


In [49]:

workflow = StateGraph(AgentState)

In [55]:
# Define validation node (updated)
def validation_node(state):
    response = state.get("response", "")
    if not response:
        print("❌ Empty response - Sending back to Supervisor")
        return {"validated": False, "response": response}
    try:
        parsed = OutputModel.model_validate({"response": response})
        print("✅ Validation Passed")
        return {"validated": True, "response": response}
    except ValidationError as e:
        print(f"❌ Validation Failed: {e}")
        return {"validated": False, "response": response}

# Add this node to the workflow
workflow.add_node("Validation", validation_node)

# Conditional routing after validation
from langgraph.graph import Send

def validation_router(state):
    if not state.get("validated"):
        return Send("Supervisor")
    else:
        return Send("Final Output")


workflow.add_node("Validation Router", validation_router)

# Now wire the edges
workflow.add_edge("LLM Call", "Validation")
workflow.add_edge("RAG", "Validation")
workflow.add_edge("Web Crawler", "Validation")
workflow.add_edge("Validation", "Validation Router")
workflow.add_edge("Validation Router", "Final Output")
workflow.add_edge("Validation Router", "Supervisor")  # fallback


ValueError: Node `Validation` already present.

In [56]:
app = workflow.compile()

ValueError: Found edge starting at unknown node 'LLM Call'

In [52]:
from IPython.display import Image,display

In [57]:
display(Image(app.get_graph().draw_mermaid_png()))

NameError: name 'app' is not defined

#### Testing

In [137]:
state={"messages":["Hi"]}

In [138]:
workflow.compile().invoke(state)

Question: Hi
Parsed response: Topic='LLM'
->-> Router ->->
Routing topic: llm
->-> LLM Call ->->
-> RAG Call ->
->-> Web Search Call ->->
Search Result: The leading online auction platform. Sell, search, bid, and win on Antiques, Collectibles, Coins, Estate & Personal Property, Cars & Trucks, Toys and more. Learn how to greet people in different situations with various expressions, from formal to funny. Discover the origin, meaning and usage of the word "hi" and its alternatives in English. Hawaii (/ h ə ˈ w aɪ. i / ⓘ hə-WY-ee; [11] Hawaiian: Hawaiʻi [həˈvɐjʔi, həˈwɐjʔi]) is an island state of the United States, in the Pacific Ocean about 2,000 miles (3,200 km) southwest of the U.S. mainland. One of the two non-contiguous U.S. states (along with Alaska), it is the only state not on the North American mainland, the only state that is an archipelago, and the only ... The best way to meet is to start with a greeting. And the most accurate way is by saying a simple 'Hi!". However, it might

InvalidUpdateError: Expected dict, got Supervisor
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_GRAPH_NODE_RETURN_VALUE

In [ ]:
state={"messages":["What are genetic diseases"]}